## Authenticate and configure API client
- this notebook demonstrates how to authenticate and configure an ApiClient for use with the vcell-rest service
- the vcell-rest service is a quarkus service that uses keycloak for authentication and authorization

## invokes login_interactive to create an authenticated ApiClient
- this will open a browser window to login to keycloak (login with alice/alice or bob/bob)
- after login, the browser will redirect to a localhost url - this is expected and can be closed
- the login_interactive method will then extract the auth code and exchange it for tokens to authenticate the api client
- the api client will be configured to use the access_token as a bearer token.

In [1]:
%env OAUTHLIB_INSECURE_TRANSPORT=1;  # allow http (not https) for local testing

from vcell_client.auth.auth_utils import login_interactive

api_url: str = "http://localhost:9000"  # local vcell-rest service - use `quarkus dev` in vcell-rest folder to start
client_id: str = 'backend-service' # default client id defined in keycloak for quarkus dev services
auth_url: str = 'https://localhost:8543/realms/quarkus/protocol/openid-connect/auth'

display("try loging in with alice/alice - has admin role")
api_client = login_interactive(api_base_url=api_url, client_id=client_id, auth_url=auth_url)

env: OAUTHLIB_INSECURE_TRANSPORT=1;  # allow http (not https) for local testing


'try loging in with alice/alice - has admin role'

192.168.1.156 - - [08/Dec/2023 23:08:27] "GET /oidc_test_callback?state=AvN6Fb0NjmhkPMnYkYppf2tUl4cgoR&session_state=50ed2bb5-94d0-47a1-a241-f9d2d692f3fd&code=dd0f3e57-0260-411f-b78c-b4caba277395.50ed2bb5-94d0-47a1-a241-f9d2d692f3fd.0ac5df91-e044-4051-bd03-106a3a5fb9cc HTTP/1.1" 200 -


### Test Publication API
- get publications - doesn't need authentication/authorization
- add publication - needs admin role (alice but not bob has this role)
- delete publication - needs admin role (alice but not bob has this role)

In [3]:
from vcell_client import Publication
from vcell_client.api.publication_resource_api import PublicationResourceApi
from datetime import date
import pandas as pd  # for pretty printing only

# access resource-specific api (all such api objects can use the same api_client)
publication_api = PublicationResourceApi(api_client)

# get all publications (expecting [] unless other publications have been added outside this notebook)
initial_pubs: list[Publication] = publication_api.get_publications()
display(f"initial publications: {initial_pubs}")

# create two publications and add them to the database (needs admin role)
pub1 = Publication(title="publication 1", authors=["author 1", "author 2"], year=1995, citation="citation 1",
    pubmedid="pubmedid 1", doi="doi 1", endnoteid=1, url="url 1", wittid=1, biomodel_refs=[], mathmodel_refs=[], var_date=date.today())
pub2 = pub1.model_copy()
pub2.title="publication 2"
pub1_key: int = publication_api.create_publication(publication=pub1)
pub2_key: int = publication_api.create_publication(publication=pub2)

# get all publications again and pretty print them with pandas (doesn't need auth)
some_pubs: list[Publication] = publication_api.get_publications()
display(pd.DataFrame([pub.to_dict() for pub in some_pubs]))

# get one publication by id (doesn't need auth)
pub1_from_db = publication_api.get_publication_by_id(id=pub1_key)
display(pd.DataFrame([pub1_from_db.to_dict()]))

# delete the publications (needs admin role)
for pub in some_pubs:
    publication_api.delete_publication(id=pub.pub_key)

# confirm that all publications have been deleted
final_pubs: list[Publication] = publication_api.get_publications()
display(f"final publications: {final_pubs}")

'initial publications: []'

,pubKey,title,authors,year,citation,pubmedid,doi,endnoteid,url,wittid,biomodelRefs,mathmodelRefs,date
0,3,publication 1,"[author 1, author 2]",1995,citation 1,pubmedid 1,doi 1,1,url 1,1,[],[],2023-12-07
1,4,publication 2,"[author 1, author 2]",1995,citation 1,pubmedid 1,doi 1,1,url 1,1,[],[],2023-12-07


,pubKey,title,authors,year,citation,pubmedid,doi,endnoteid,url,wittid,biomodelRefs,mathmodelRefs,date
0,3,publication 1,"[author 1, author 2]",1995,citation 1,pubmedid 1,doi 1,1,url 1,1,[],[],2023-12-07


'final publications: []'